# **Faster Whisper 사용법**


[![notebook shield](https://img.shields.io/static/v1?label=&message=Notebook&color=blue&style=for-the-badge&logo=googlecolab&link=https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)](https://colab.research.google.com/github/lewangdev/whisper-youtube/blob/main/faster_whisper_youtube.ipynb)
[![repository shield](https://img.shields.io/static/v1?label=&message=Repository&color=blue&style=for-the-badge&logo=github&link=https://github.com/lewangdev/faster_whisper_youtube)](https://github.com/lewangdev/faster_whisper_youtube)


[faster-whisper](https://github.com/guillaumekln/faster-whisper) 는 트랜스포머 모델의 빠른 추론 엔진인 CTranslate2를 사용하여 OpenAI의 Whisper 모델을 재구현한 것입니다.

이 구현은 openai/whisper보다 최대 4배 빠르며 동일한 정확도와 적은 메모리를 사용합니다. CPU와 GPU 모두에서 8비트 양자화로 효율성을 더욱 향상시킬 수 있습니다.

Whisper는 범용 음성 인식 모델입니다. 다양한 오디오의 대규모 데이터 세트에서 학습되며 다국어 음성 인식은 물론 음성 번역 및 언어 식별을 수행할 수 있는 멀티태스킹 모델이기도 합니다.

이 노트북은 Faster Whisper를 사용한 YouTube 비디오의 전사 과정을 안내합니다. 대부분의 추론 매개 변수를 탐색하거나 노트북을 그대로 사용하여 녹취록과 비디오 오디오를 Google 드라이브에 저장할 수 있습니다.

*   항목 추가
*   항목 추가



In [ ]:
#@markdown # **GPU type** 🕵 ️

#@markdown Colab 세션에서 할당받는 GPU 유형은 비디오가 전사되는 속도를 정의했습니다.
#@markdown FLOPS(Floating Point Operations Per Second) 횟수가 많을수록 전사 속도가 빨라집니다.
#@markdown 그러나 Colab에서 사용할 수 있는 가장 강력하지 않은 GPU조차도 Whisper 모델을 실행할 수 있습니다.
#@markdown 노트북의 하드웨어 액셀러레이터로 'GPU'를 선택했는지 확인합니다(Runtime → Change runtime type → Hardware acceler).


#@markdown |  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
#@markdown |:------:|:----------:|:--------------:|:------------------:|
#@markdown |  T4    |    16 GB   |       8.1      |         Free       |
#@markdown | P100   |    16 GB   |      10.6      |      Colab Pro     |
#@markdown | V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

#@마크다운 ---
#@markdown **새로운 GPU를 할당받으려면 노트북의 런타임을 재설정합니다.**

!nvidia-smi -L

!nvidia-smi

In [ ]:
#@markdown # **라이브러리 설치** 🏗 ️
#@markdown 이 셀은 Faster Whisper를 포함한 여러 라이브러리를 다운로드하는 데 시간이 조금 걸립니다.


! pip install faster-whisper
! pip install yt-dlp

import sys
import warnings
from faster_whisper import WhisperModel
from pathlib import Path
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

In [ ]:
#@markdown # **Optional:** Save data in Google Drive 💾
#@markdown Enter a Google Drive path and run this cell if you want to store the results inside Google Drive.

# Uncomment to copy generated images to drive, faster than downloading directly from colab in my experience.
from google.colab import drive
drive_mount_path = Path("/") / "content" / "drive"
drive.mount(str(drive_mount_path))
drive_mount_path /= "My Drive"
#@markdown ---
drive_path = "Colab Notebooks/Faster Whisper Youtube" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change your Google Drive path.**

drive_whisper_path = drive_mount_path / Path(drive_path.lstrip("/"))
drive_whisper_path.mkdir(parents=True, exist_ok=True)

In [ ]:
#@마크다운 # **모델선택** 🧠

#@markdown 첫 번째 공개 릴리스를 기준으로 다음과 같은 4가지 사전 교육 옵션을 사용할 수 있습니다:

#@markdown |  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
#@markdown |:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
#@markdown |  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~0.8 GB     |      ~32x      |
#@markdown |  base  |    74 M    |     `base.en`      |       `base`       |     ~1.0 GB     |      ~16x      |
#@markdown | small  |   244 M    |     `small.en`     |      `small`       |     ~1.4 GB     |      ~6x       |
#@markdown | medium |   769 M    |    `medium.en`     |      `medium`      |     ~2.7 GB     |      ~2x       |
#@markdown | large-v1  |   1550 M   |        N/A         |      `large-v1`       |    ~4.3 GB     |       1x       |
#@markdown | large-v2  |   1550 M   |        N/A         |      `large-v2`       |    ~4.3 GB     |       1x       |

#@markdown ---
model_size = 'large-v2' #@param ['tiny', 'tiny.en', 'base', 'base.en', 'small', 'small.en', 'medium', 'medium.en', 'large-v1', 'large-v2']
device_type = "cuda" #@param {type:"string"} ['cuda', 'cpu']
compute_type = "float16" #@param {type:"string"} ['float16', 'int8_float16', 'int8']
#@markdown ---
#@markdown **모델을 변경하면 이 셀을 다시 실행합니다.**

model = WhisperModel(model_size, device=device_type, compute_type=compute_type)


In [17]:
#@markdown # **Video selection** 📺

#@markdown Enter the URL of the Youtube video you want to transcribe, wether you want to save the audio file in your Google Drive, and run the cell.

Type = "Google Drive" #@param ['Youtube video or playlist', 'Google Drive']
#@markdown ---
#@markdown #### **Youtube video or playlist**
URL = "https://www.youtube.com/watch?v=7oNMH6HUhk8" #@param {type:"string"}
# store_audio = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **Google Drive video, audio (mp4, wav), or folder containing video and/or audio files**
video_path = "SPK003YTNSO162/SPK003YTNSO162F001.wav" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change the video.**

video_path_local_list = []

if Type == "Youtube video or playlist":

    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',
        # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
        'postprocessors': [{  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([URL])
        list_video_info = [ydl.extract_info(URL, download=False)]

    for video_info in list_video_info:
        video_path_local_list.append(Path(f"{video_info['id']}.wav"))

elif Type == "Google Drive":
    # video_path_drive = drive_mount_path / Path(video_path.lstrip("/"))
    video_path = drive_mount_path / Path(video_path.lstrip("/"))
    if video_path.is_dir():
        for video_path_drive in video_path.glob("**/*"):
            if video_path_drive.is_file():
                display(Markdown(f"**{str(video_path_drive)} selected for transcription.**"))
            elif video_path_drive.is_dir():
                display(Markdown(f"**Subfolders not supported.**"))
            else:
                display(Markdown(f"**{str(video_path_drive)} does not exist, skipping.**"))
            video_path_local = Path(".").resolve() / (video_path_drive.name)
            shutil.copy(video_path_drive, video_path_local)
            video_path_local_list.append(video_path_local)
    elif video_path.is_file():
        video_path_local = Path(".").resolve() / (video_path.name)
        shutil.copy(video_path, video_path_local)
        video_path_local_list.append(video_path_local)
        display(Markdown(f"**{str(video_path)} selected for transcription.**"))
    else:
        display(Markdown(f"**{str(video_path)} does not exist.**"))

else:
    raise(TypeError("Please select supported input type."))

for video_path_local in video_path_local_list:
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
        result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])


**/content/drive/My Drive/SPK003YTNSO162/SPK003YTNSO162F001.wav selected for transcription.**

In [18]:
def seconds_to_time_format(s):
    # Convert seconds to hours, minutes, seconds, and milliseconds
    hours = s // 3600
    s %= 3600
    minutes = s // 60
    s %= 60
    seconds = s // 1
    milliseconds = round((s % 1) * 1000)

    # Return the formatted string
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds):03d}"

#@markdown # **모델 실행** 🚀

#@markdown 비디오의 전사를 실행하기 위해 이 셀을 실행합니다. 비디오의 길이와 위에서 선택한 모델의 매개 변수 수에 따라 시간이 많이 걸릴 수 있습니다.

#@markdown ## **Parameters** ⚙ ️

#@markdown ### **행동제어**
#@markdown ####언어
language = "auto" #@param ["auto", "en", "zh", "ja", "fr", "de"] {allow-input: true}
#@markdown ##### 초기 프롬프트

initial_prompt = "Please do not translate, only transcription be allowed.  Here are some English words you may need: Cindy. And Chinese words: \u7206\u7834" #@param {type:"string"}
#@markdown ---
#@markdown #### Word-level timestamps
word_level_timestamps = False #@param {type:"boolean"}
#@markdown ---
#@markdown #### VAD filter
vad_filter = True #@param {type:"boolean"}
vad_filter_min_silence_duration_ms = 50 #@param {type:"integer"}
#@markdown ---
#@markdown #### Output(Default is srt, txt if `text_only` be checked )
text_only = False #@param {type:"boolean"}


segments, info = model.transcribe(str(video_path_local), beam_size=5,
                                  language=None if language == "auto" else language,
                                  initial_prompt=initial_prompt,
                                  word_timestamps=word_level_timestamps,
                                  vad_filter=vad_filter,
                                  vad_parameters=dict(min_silence_duration_ms=vad_filter_min_silence_duration_ms))

display(Markdown(f"Detected language '{info.language}' with probability {info.language_probability}"))

ext_name = '.txt' if text_only else ".srt"
transcript_file_name = video_path_local.stem + ext_name
sentence_idx = 1
with open(transcript_file_name, 'w') as f:
  for segment in segments:
    if word_level_timestamps:
      for word in segment.words:
        ts_start = seconds_to_time_format(word.start)
        ts_end = seconds_to_time_format(word.end)
        print(f"[{ts_start} --> {ts_end}] {word.word}")
        if not text_only:
          f.write(f"{sentence_idx}\n")
          f.write(f"{ts_start} --> {ts_end}\n")
          f.write(f"{word.word}\n\n")
        else:
          f.write(f"{word.word}")
        f.write("\n")
        sentence_idx = sentence_idx + 1
    else:
      ts_start = seconds_to_time_format(segment.start)
      ts_end = seconds_to_time_format(segment.end)
      print(f"[{ts_start} --> {ts_end}] {segment.text}")
      if not text_only:
        f.write(f"{sentence_idx}\n")
        f.write(f"{ts_start} --> {ts_end}\n")
        f.write(f"{segment.text.strip()}\n\n")
      else:
        f.write(f"{segment.text.strip()}\n")
      sentence_idx = sentence_idx + 1

try:
  shutil.copy(video_path_local.parent / transcript_file_name,
            drive_whisper_path / transcript_file_name
  )
  display(Markdown(f"**Transcript file created: {drive_whisper_path / transcript_file_name}**"))
except:
  display(Markdown(f"**Transcript file created: {video_path_local.parent / transcript_file_name}**"))


Detected language 'ko' with probability 0.99951171875

[00:00:00,000 --> 00:00:12,000]  이른바 햄버거병 유발 가능성이 있는 햄버거용 패티를 맥도날드에 납품한 혐의로 영장이 청구된 업체의 임원들이 또 구속위기를 피했습니다.


**Transcript file created: /content/drive/My Drive/Colab Notebooks/Faster Whisper Youtube/SPK003YTNSO162F001.srt**